In [1]:
import os
os.sys.path

['C:\\Users\\sride\\Downloads',
 'C:\\Users\\sride\\anaconda3\\python38.zip',
 'C:\\Users\\sride\\anaconda3\\DLLs',
 'C:\\Users\\sride\\anaconda3\\lib',
 'C:\\Users\\sride\\anaconda3',
 '',
 'C:\\Users\\sride\\anaconda3\\lib\\site-packages',
 'C:\\Users\\sride\\anaconda3\\lib\\site-packages\\locket-0.2.1-py3.8.egg',
 'C:\\Users\\sride\\anaconda3\\lib\\site-packages\\win32',
 'C:\\Users\\sride\\anaconda3\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\sride\\anaconda3\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\sride\\anaconda3\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\sride\\.ipython']

In [2]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install easygui


Note: you may need to restart the kernel to use updated packages.


In [4]:
import cv2 #for image processing
import easygui #to open the filebox
import numpy as np #to store image
import imageio #to read image stored at particular path

In [5]:
import sys
import matplotlib.pyplot as plt
import os
import tkinter as tk
from tkinter import filedialog
from tkinter import *
from PIL import ImageTk, Image

In [6]:
top=tk.Tk()
top.geometry('400x400')
top.title('Cartoonify Your Image !')
top.configure(background='white')
label=Label(top,background='#CDCDCD', font=('calibri',20,'bold'))

In [7]:
def onlive():
    switch(1)

In [8]:
def upload():
    ImagePath=easygui.fileopenbox()
    cartoonify(ImagePath)

In [9]:
def cartoonify(ImagePath):
    # read the image
    originalmage = cv2.imread(ImagePath)
    originalmage = cv2.cvtColor(originalmage, cv2.COLOR_BGR2RGB)
    #print(image)  # image is stored in form of numbers

    # confirm that image is chosen
    if originalmage is None:
        print("Can not find any image. Choose appropriate file")
        sys.exit()

    ReSized1 = cv2.resize(originalmage, (960, 540))
    #plt.imshow(ReSized1, cmap='gray')


    #converting an image to grayscale
    grayScaleImage= cv2.cvtColor(originalmage, cv2.COLOR_BGR2GRAY)
    ReSized2 = cv2.resize(grayScaleImage, (960, 540))
    #plt.imshow(ReSized2, cmap='gray')


    #applying median blur to smoothen an image
    smoothGrayScale = cv2.medianBlur(grayScaleImage, 5)
    ReSized3 = cv2.resize(smoothGrayScale, (960, 540))
    #plt.imshow(ReSized3, cmap='gray')

    #retrieving the edges for cartoon effect
    #by using thresholding technique
    getEdge = cv2.adaptiveThreshold(smoothGrayScale, 255, 
        cv2.ADAPTIVE_THRESH_MEAN_C, 
        cv2.THRESH_BINARY, 9, 9)

    ReSized4 = cv2.resize(getEdge, (960, 540))
    #plt.imshow(ReSized4, cmap='gray')

    #applying bilateral filter to remove noise 
    #and keep edge sharp as required
    colorImage = cv2.bilateralFilter(originalmage, 9, 300, 300)
    ReSized5 = cv2.resize(colorImage, (960, 540))
    #plt.imshow(ReSized5, cmap='gray')
   


    #masking edged image with our "BEAUTIFY" image
    cartoonImage = cv2.bitwise_and(colorImage, colorImage, mask=getEdge)

    ReSized6 = cv2.resize(cartoonImage, (960, 540))
    #plt.imshow(ReSized6, cmap='gray')

    # Plotting the whole transition
    images=[ReSized1, ReSized2, ReSized3, ReSized4, ReSized5, ReSized6]

    fig, axes = plt.subplots(3,2, figsize=(8,8), subplot_kw={'xticks':[], 'yticks':[]}, gridspec_kw=dict(hspace=0.1, wspace=0.1))
    for i, ax in enumerate(axes.flat):
        ax.imshow(images[i], cmap='gray')

    save1=Button(top,text="Save cartoon image",command=lambda: save(ReSized6, ImagePath),padx=30,pady=5)
    save1.configure(background='#364156', foreground='white',font=('calibri',10,'bold'))
    save1.pack(side=TOP,pady=50)
    
    plt.show()
    

In [10]:
def save(ReSized6, ImagePath):
    #saving an image using imwrite()
    newName="cartoonified_Image"
    path1 = os.path.dirname(ImagePath)
    extension=os.path.splitext(ImagePath)[1]
    path = os.path.join(path1, newName+extension)
    cv2.imwrite(path, cv2.cvtColor(ReSized6, cv2.COLOR_RGB2BGR))
    I= "Image saved by name " + newName +" at "+ path
    tk.messagebox.showinfo(title=None, message=I)






In [11]:
load=Button(top,text="Cartoonify an Image",command=upload,padx=10,pady=5)
live=Button(top,text="Live Cartoonify",command=onlive,padx=10,pady=5)
load.configure(background='#364156', foreground='white',font=('calibri',10,'bold'))
live.configure(background='#364156', foreground='white',font=('calibri',10,'bold'))
live.pack(side=TOP,pady=20)
load.pack(side=TOP,pady=20)


In [12]:
def liv():
    cap=cv2.VideoCapture(0)
    while(1):
        ret, frame =cap.read()
        cv2.imshow('the_input_frame',frame)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        edges = cv2.Canny(frame,100,10)
        cv2.imshow('edges',edges)

        color = cv2.bilateralFilter(frame, d=7, sigmaColor=6,sigmaSpace=9)
        def color_quantization(img, k):
    # Defining input data for clustering
              data = np.float32(img).reshape((-1, 3))
    # Defining criteria
              criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 20, 1.0)
    # Applying cv2.kmeans function
              ret, label, center = cv2.kmeans(data, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
              center = np.uint8(center)
              result = center[label.flatten()]
              result = result.reshape(img.shape)
              return result

        img_1 = color_quantization(frame, 2)
        cartoon = cv2.bitwise_and(color, color, img_1,mask=edges)
        cv2.imshow('cartoon',cartoon)
    #     edges=cv2.Canny(frame,100,100)
        k=cv2.waitKey(5) & 0xFF
        if k== 27:
            break
    cap.release()


def up():
    top.mainloop()

def default():
    return "choose correct option"
   
switcher = {
    1: liv,
    2: up
    }

def switch(value):
    return switcher.get(value, default)()
switch(2)